In [2]:
import pandas as pd
from datetime import datetime, timedelta, date
import numpy as np
CP = "0303659P"

# path = "C:/Users/" + CP+ "/SNCF/Pôle data - Lab' MTA GrpO365 - Documents/Projets/Hackathon/Données/Données brutes"
path = "C:/Users/0303659P/OneDrive - SNCF/Documents/Course_et_comptage_Ligne_H_2023.csv"
df = pd.read_csv(path)


In [25]:
start_date = datetime.strptime("2023-11-01", "%Y-%m-%d")
end_date = datetime.strptime("2023-11-15", "%Y-%m-%d")

In [26]:
df_trains = df[["Date",'Heure_départ_réalisé','Code_gare','Mission']]
df_trains = df_trains[df_trains.Mission.isin(["ADDO","ADEO","APOR","ODDA","ODEA","OPOC"])]
df_trains["Sens"] = df_trains["Mission"].apply(lambda x : "Vers Paris" if x in (["ADDO","ADEO","APOR"]) else "Vers province")
df_trains['Heure_départ_réalisé'] = pd.to_datetime(df_trains['Heure_départ_réalisé']).dt.tz_localize(None)


df_trains = df_trains.drop(["Mission"],axis=1)

df_trains["Date"] = pd.to_datetime(df_trains["Date"])

df_trains = df_trains[(df_trains["Heure_départ_réalisé"].dt.hour >= 6) & (df_trains["Heure_départ_réalisé"].dt.hour < 23)]

df_trains = df_trains[(df_trains["Date"] <= end_date) & (df_trains["Date"] >= start_date)]

In [21]:
#vers Paris: ADDO ADEO APOR
#vers banlieue: ODDA ODEA OPOC

In [28]:
# Liste des gares
gares = df_trains.Code_gare.unique()

time_delta = timedelta(minutes=5)
date_delta = timedelta(days=1)

heures_minutes = []
gares_list = []

for gare in gares:
    current_date = start_date
    while current_date <= end_date:
        start_time = current_date.replace(hour=6, minute=0)
        end_time = current_date.replace(hour=23, minute=55)
        current_time = start_time
        while current_time <= end_time:
            heures_minutes.append(current_time)
            gares_list.append(gare)
            current_time += time_delta
        current_date += date_delta

# Créer le DataFrame
data = {
    "Gare": gares_list,
    "Date": heures_minutes
}

df_output = pd.DataFrame(data)
df_output["Date"] = pd.to_datetime(df_output["Date"])

In [29]:
# Calculer le temps d'attente pour le prochain train
def calculer_temps_attente_vers_paris(row):
    date = row["Date"]
    jour = row["Date"].date()
    gare = row["Gare"]
    prochains_trains_vers_paris = df_trains_travail[(df_trains_travail['Date'].dt.date == jour)
                                            & (df_trains_travail['Sens'] == "Vers Paris")
                                            & (df_trains_travail['Heure_départ_réalisé'] > date)
                                            & (df_trains_travail["Code_gare"] == gare)]
    if not prochains_trains_vers_paris.empty:
        prochain_train_vers_paris = prochains_trains_vers_paris['Heure_départ_réalisé'].min()
        temps_attente_vers_paris = (prochain_train_vers_paris - date).total_seconds() / 60  # en minutes
        return temps_attente_vers_paris
    else:
        return np.nan

def calculer_temps_attente_vers_province(row):
    date = row["Date"]
    jour = row["Date"].date()
    gare = row["Gare"]
    prochains_trains_vers_province = df_trains_travail[(df_trains_travail['Date'].dt.date == jour)
                                            & (df_trains_travail['Sens'] == "Vers province")
                                            & (df_trains_travail['Heure_départ_réalisé'] > date)
                                            & (df_trains_travail["Code_gare"] == gare)]
    if not prochains_trains_vers_province.empty:
        prochain_train_vers_province = prochains_trains_vers_province['Heure_départ_réalisé'].min()
        temps_attente_vers_province = (prochain_train_vers_province - date).total_seconds() / 60  # en minutes
        return temps_attente_vers_province
    else:
        return np.nan

jours = df_trains.Date.dt.date.unique()

df_output_final = df_output.iloc[0:0].copy()

for jour in jours[:]:
    df_trains_travail = df_trains[df_trains.Date.dt.date == jour]
    df_output_travail_vers_paris = df_output[df_output.Date.dt.date == jour]
    df_output_travail_vers_province = df_output[df_output.Date.dt.date == jour]

    df_output_travail_vers_province["temps_attente"] = df_output_travail_vers_province.apply(calculer_temps_attente_vers_province, axis=1)
    df_output_travail_vers_province["Sens"] = "Vers province"
    df_output_travail_vers_paris["temps_attente"] = df_output_travail_vers_paris.apply(calculer_temps_attente_vers_paris, axis=1)
    df_output_travail_vers_paris["Sens"] = "Vers Paris"

    df_output_travail = pd.concat([df_output_travail_vers_paris,df_output_travail_vers_province], axis=0)
    
    df_output_final = pd.concat([df_output_final,df_output_travail],axis=0)

C:\Users\0303659P\AppData\Local\Temp\ipykernel_32536\876543537.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_output_travail_vers_province["temps_attente"] = df_output_travail_vers_province.apply(calculer_temps_attente_vers_province, axis=1)
C:\Users\0303659P\AppData\Local\Temp\ipykernel_32536\876543537.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_output_travail_vers_province["Sens"] = "Vers province"
C:\Users\0303659P\AppData\Local\Temp\ipykernel_32536\876543537.py:43: SettingWithCopyWa

In [30]:
df_output_final.to_excel("../Données/Données pré-traitées/database_temps_attente.xlsx")

In [32]:
display(df_output_final)

,Gare,Date,temps_attente,Sens
2592,FPB,2023-11-13 06:00:00,9.000000,Vers Paris
2593,FPB,2023-11-13 06:05:00,4.000000,Vers Paris
2594,FPB,2023-11-13 06:10:00,13.963733,Vers Paris
2595,FPB,2023-11-13 06:15:00,8.963733,Vers Paris
2596,FPB,2023-11-13 06:20:00,3.963733,Vers Paris
...,...,...,...,...
39523,PSE,2023-11-03 23:35:00,NaN,Vers province
39524,PSE,2023-11-03 23:40:00,NaN,Vers province
39525,PSE,2023-11-03 23:45:00,NaN,Vers province
39526,PSE,2023-11-03 23:50:00,NaN,Vers province
